# Camera Predictions

In [ ]:
!pip install -qr requirements.txt

In [52]:
import io
import boto3
import numpy as np
import pandas as pd
from PIL import Image
import skimage

In [2]:
s3 = boto3.resource("s3")
bucket = s3.Bucket("ap-unsplash-images")

In [4]:
with io.BytesIO() as b:
    bucket.download_fileobj("lite-normalized-photos.csv",b)
    b.seek(0)
    df = pd.read_csv(b)
df.head()

,photo_id,photo_image_url,photo_width,photo_height,photo_aspect_ratio,exif_camera_make,exif_camera_model,exif_iso,exif_aperture_value,exif_focal_length,exif_exposure_time,s3_key,s3_bucket
0,2Q8zDWkj0Yw,https://images.unsplash.com/photo-141520117961...,4192,2794,1.50,NIKON CORPORATION,NIKON D700,2000.0,2.5,50.0,1/125,lite-normalized/unsplash.lite.2Q8zDWkj0Yw.JPG,ap-unsplash-images
1,tsBDNuCJiLg,https://images.unsplash.com/photo-141768928330...,4324,2880,1.50,NIKON CORPORATION,NIKON D3200,320.0,7.1,200.0,1/2000,lite-normalized/unsplash.lite.tsBDNuCJiLg.JPG,ap-unsplash-images
2,A93gsuMxVcE,https://images.unsplash.com/photo-142981401899...,2000,1333,1.50,Canon,Canon EOS REBEL T2i,400.0,14.0,18.0,1/640,lite-normalized/unsplash.lite.A93gsuMxVcE.JPG,ap-unsplash-images
3,oYIdH6bFssk,https://images.unsplash.com/photo-143275722183...,4288,2848,1.51,NIKON CORPORATION,NIKON D5000,250.0,5.6,105.0,1/30,lite-normalized/unsplash.lite.oYIdH6bFssk.JPG,ap-unsplash-images
4,wgLPy2YBXuc,https://images.unsplash.com/photo-143205996405...,5312,2988,1.78,SAMSUNG,SAMSUNG-SM-G870A,40.0,2.2,4.8,1/2384,lite-normalized/unsplash.lite.wgLPy2YBXuc.JPG,ap-unsplash-images


In [5]:
bucket.download_file(df.loc[0,"s3_key"],"data/test-img.jpg")

# Predicting Camera Focal Length

In [23]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

from sklearn.model_selection import train_test_split

In [40]:
class FoculLengthDataset(Dataset):
    def __init__(self, df):
        # Connect to S3
        s3 = boto3.resource("s3")
        self.s3_bucket = s3.Bucket("ap-unsplash-images")
        # Store the DataFrame
        self.df = df.reset_index(drop=True)[["s3_key","exif_focal_length"]]
        # Build transforms
        self.transforms = transforms.Compose([
            transforms.Resize((512,512)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.56040615,0.36607313,0.42010993],
                std=[0.2895784,0.23586935,0.22863376]
            )
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,i):
        label = self.df.loc[i,"exif_focal_length"]
        s3key = self.df.loc[i,"s3_key"]
        with io.BytesIO() as b:
            self.s3_bucket.download_fileobj(s3key,b)
            b.seek(0)
            tens = self.transforms(Image.open(b))
        return label, tens

In [41]:
indexes = np.arange(len(df))
i_train, i_val = train_test_split(indexes,test_size=0.15)

train_dataset = FoculLengthDataset(df.iloc[i_train])
val_dataset = FoculLengthDataset(df.iloc[i_val])

In [42]:
BATCH_SIZE = 64

In [43]:
train_loader = DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE)
val_loader = DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE)

In [44]:
train_dataset[0]

AttributeError: module 'PIL.TiffTags' has no attribute 'IFD'

In [54]:
from skimage import io as imgio

In [55]:
# train_dataset.transforms()
img = imgio.imread("data/test-img.jpg")
img

AttributeError: module 'PIL.TiffTags' has no attribute 'IFD'